In [ ]:
import scipy.sparse
import sklearn
from sklearn import preprocessing
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, accuracy_score
#from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.metrics import classification_report
import pandas as pd
import optuna
import xgboost as xgb

In [ ]:
mytrain = pd.read_csv('../input/tabular-playground-series-may-2021/train.csv')
mytest = pd.read_csv('../input/tabular-playground-series-may-2021/test.csv')

In [ ]:
mytrain.head()

In [ ]:
mytest.head()

In [ ]:
big_X = mytrain.drop(['id', 'target'], axis=1)
small_y = mytrain.target
le = LabelEncoder() # practice, i could do it manually if i wanted... 
small_y = le.fit_transform(small_y)

X_test = mytest.drop(['id'], axis=1) 

In [ ]:
def objective(trial):
    
    X_train, X_val, y_train, y_val = train_test_split(big_X, 
                    small_y, test_size=0.2,random_state=42)
    
    params = {"verbosity":0,
              
    #'tree_method':'gpu_hist', #not exact for now 
    
    # i dont know if i need:
    #'predictor':'gpu_predictor', 
              
    "objective": "multi:softprob",  
    # at one point i actually used softmax !  
              
    "num_class":4, 

    'eval_metric': 'mlogloss',
    # keep ? 'metric': 'multi_logloss', 
                 
    'booster' : 'gbtree',
    # not this: "boosting": 'gbtree', 
    
    'random_state':42, 
              
    # "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
    'reg_lambda' : trial.suggest_loguniform('reg_lambda' , 1e-8 , 30), 
    "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 1.0, log=True),
    # 'reg_alpha':trial.suggest_int('reg_alpha', 0, 5),
    # 'reg_lambda':trial.suggest_int('reg_lambda', 0, 5),
             
    "subsample": trial.suggest_float("subsample", 0.2, 1.0, step=.1),
    # 'subsample' : 0.8,
             
    "colsample_bytree": trial.suggest_discrete_uniform("colsample_bytree",0.5,1,0.1),
    # no: 'colsample_bytree' : trial.suggest_uniform('colsample_bytree',0,1),
             
    "max_depth": trial.suggest_int("max_depth", 3, 25, step=1),
             
    "min_child_weight": trial.suggest_int("min_child_weight", 1, 100), 
              
    # "eta":  trial.suggest_float("eta", 1e-8, 1.0, log=True), 
    # 'learning_rate':trial.suggest_loguniform('learning_rate',0.005,0.5),
    'learning_rate' : trial.suggest_uniform('learning_rate' , 0.005 , 0.3),
    'gamma' : trial.suggest_loguniform('gamma' , 1e-8 , 1.0),           
    "grow_policy": trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"]),
    'n_estimators' : trial.suggest_int('n_estimators' , 100, 2500),
    'max_leaves': trial.suggest_int('max_leaves', 2, 40)
            } 
      
    earlyStop=100
    

    bst = xgb.XGBClassifier(**params)
    
    bst.fit(X_train, y_train, eval_set = [(X_val,y_val)], 
            early_stopping_rounds = 100, verbose = False)
    
    y_pred = bst.predict_proba(X_val)
        
    return sklearn.metrics.log_loss(y_val, y_pred)

In [ ]:
# study = optuna.create_study(direction="minimize")
# study.optimize(objective, n_trials=100) 

# print("Number of finished trials: ", len(study.trials))
# print("Best trial:")
# trial = study.best_trial

# print("  Value: {}".format(trial.value))
# print("  Params: ")
# for key, value in trial.params.items():
#     print("    {}: {}".format(key, value))

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-may-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-may-2021/test.csv')
sample_submission = pd.read_csv('../input/tabular-playground-series-may-2021/sample_submission.csv') 
le = LabelEncoder() # practice, i could do it manually if i wanted... 
train.target = le.fit_transform(train.target)
features = train.columns[1:51] # only fifty 
X_train, X_val, y_train, y_val = train_test_split(train.drop(columns=["id","target"]), train.target, stratify=train.target)
# y_train = y_train.map({'Class_1':0, 
#                        'Class_2':1, 
#                        'Class_3':2, 
#                        'Class_4':3})

X_test = test.drop(['id'], axis=1) 

bst_2 = xgb.XGBClassifier(seed=42, 
base_score=0.5,
booster='gbtree',
#reg_lambda = 0.6838907631827036,  
# , 
#reg_alpha =  0.004407099400692902,  
# , 
colsample_bytree = 0.5, 
max_depth = 3, 
min_child_weight = 87, 
learning_rate = 0.038142105877918005, 
#scale_pos_weight=2,
gamma =0.0021897234762554415, 
                          
#grow_policy = "lossguide", 
n_estimators = 1409, 
num_class=4,                
subsample=0.7,
max_leaves=12, 
importance_type='gain', 
interaction_constraints='', 
monotone_constraints='()',
#num_parallel_tree=1,
verbosity=1, 
scale_pos_weight=0.5,
#tree_method = 'gpu_hist',  
objective = "multi:softprob")
# eval_metric = 'mlogloss'
# should be down below, not here
                                           
bst_2.fit(
X_train, 
y_train,
verbose=True,
early_stopping_rounds=30,
eval_metric=['mlogloss'],
eval_set=[(X_train, y_train),(X_val, y_val)])

In [ ]:
preds_sub = bst_2.predict_proba(X_test)
classes = ["Class_1","Class_2","Class_3","Class_4"]
sample_submission = pd.read_csv('../input/tabular-playground-series-may-2021/sample_submission.csv') 
sample_submission.drop(columns=classes,inplace=True)
submission = (sample_submission.join(pd.DataFrame(data=preds_sub, 
                                    columns=classes)))
submission.to_csv("my_submission.csv", index=False)